# Visualization

- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
- 30% of the company staff have at least 1 child.
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
- Account managers need to travel a lot.
- Everyone in the company is between 25 and 40, give them some place to go party.
- The CEO is vegan.
- If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
- The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.

In [40]:
from dotenv import load_dotenv
import os
import requests
import geopandas as gpd
import pandas as pd
from cartoframes.viz import Map, Layer, popup_element, basemaps
from cartoframes.auth import Credentials
from cartoframes.auth import set_default_credentials
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import glob

In [257]:
companies = pd.read_csv("../data/companies_cleaned.csv")
load_dotenv()
token_fsq = os.getenv("token_fsq")

In [3]:
companies

,Name,Number of employees,Description,Total money (in million $),Main address,City,Latitude,Longitude
0,ngmoco,150,Games for the iPhone,44.8,185 Berry Street,San Francisco,37.770510,-122.401581
1,Thumbplay,70,"Music, Videos, Games for Mobile Devices",41.5,599 Broadway,New York,40.725435,-73.997395
2,OMGPOP,50,Free online multiplayer game,16.6,SoHo,New York,40.723384,-74.001704
3,CyberArts,55,Gaming software developer,16.3,"51 Arbor St,",San Francisco,37.737609,-122.435399
4,Zynga,115,Network of gaming applications,8.0,365 Vermont St.,San Francisco,37.765158,-122.404234
5,Serious Business,22,Social Game Development Studio,4.0,580 Market Street,San Francisco,37.789321,-122.401362
6,Exent,100,Games on Demand Leader,3.0,632 W. 28th St.,New York,40.752380,-74.005568
7,Exent,100,Games on Demand Leader,3.0,685 Market Street,San Francisco,37.787646,-122.402759
8,GameLayers,5,Multiplayer Networked Games,2.0,1 Lombard Street,San Francisco,37.803847,-122.402468
9,Shattered Reality Interactive,10,"Online Social Gaming, MMO",1.1,490 2nd Street,San Francisco,37.783046,-122.394067


In [253]:
def call_FSQ(category):
    
    categories= {"vegan" : 13377,
                 "daycare" : 11026,
                 "night club" : 10032,
                 "airport" : 19031,
                 "bus" : 19042,
                 "metro" : 19046,
                 "train" : 19047,
                 "tram" : 19050,
                 "basket" : 18006
                }
    lst=[]
    
    for i in range(0,13):
              
        categ = categories[f"{category}"]
        latitude = companies.iloc[i].loc['Latitude']
        longitude = companies.iloc[i].loc['Longitude']

    
        url = f"https://api.foursquare.com/v3/places/search?ll={latitude}%2C{longitude}&categories={categ}&radius=2000&limit=10"

        headers = {"accept": "application/json",
                  "Authorization" : token_fsq}

        response = requests.get(url, headers=headers).json()

        for i in response["results"]:
            name = i["name"]
            distance = i["distance"]
            address =  i["location"]["formatted_address"]
            lat = i["geocodes"]["main"]["latitude"]
            long = i["geocodes"]["main"]["longitude"]
            type_ = {"typepoint": 
                                {"type": "Point", 
                                "coordinates": [lat, long]}}

            lst.append({"name":name, "lat":lat, "lon":long, "distance":distance, "type":type_})

    category = pd.DataFrame(lst)
    category = gpd.GeoDataFrame(category, geometry=gpd.points_from_xy(category["lon"], category["lat"]))
    
    mapa = Map(Layer(category, "color:purple", popup_hover=[popup_element("name", "Restaurants near each location")]), basemap=basemaps.voyager)

    return mapa


In [325]:
def get_tables(row, category, radius):
    
    categories= {"vegan" : 13377,
                 "daycare" : 11026,
                 "night club" : 10032,
                 "airport" : 19037,
                 "bus" : 19042,
                 "metro" : 19046,
                 "train" : 19047,
                 "tram" : 19050,
                 "basket" : 18008
                }
    
    queries= ["vegan restaurant", ""]

    latitude = companies.iloc[row].loc['Latitude']
    longitude = companies.iloc[row].loc['Longitude']
    categ = categories[category]

    url = f"https://api.foursquare.com/v3/places/search?ll={latitude}%2C{longitude}&categories={categ}&radius={radius}&limit=10"

    headers = {"accept": "application/json",
                          "Authorization" : token_fsq}

    response = requests.get(url, headers=headers).json()

    lst=[]
    
    for i in response["results"]:
        name = i["name"]
        distance = i["distance"]
        address =  i["location"]["formatted_address"]
        lat = i["geocodes"]["main"]["latitude"]
        long = i["geocodes"]["main"]["longitude"]
        type_ = {"typepoint": 
                            {"type": "Point", 
                            "coordinates": [lat, long]}}

        lst.append({"name":name, "lat":lat, "lon":long, "distance":distance, "type":type_})

    df = pd.DataFrame(lst)
    df.to_csv(f"../data/{row}{category}.csv", index=False)
        
    pass

In [326]:
# FUNCTION TO EXTRACT ALL SINGLE CATEGORIES
def extract_categories(radius):
    
    categories= {"vegan" : 13377,
                 "daycare" : 11026,
                 "night club" : 10032,
                 "airport" : 19037,
                 "bus" : 19042,
                 "metro" : 19046,
                 "train" : 19047,
                 "tram" : 19050,
                 "basket" : 18008
                    }
    for i in range(len(companies)):
        for j in categories.keys():
            get_table(i, j, radius)
    pass

In [206]:
# FUNCTION TO APPEND ALL CATEGORIES IN A SINGLE DATAFRAME
lst=[]
  
path = os.getcwd()
csv_files = glob.glob(os.path.join("../data/", "1*.csv"))
  
for f in csv_files:
    if os.path.getsize(f) > 100:
        df = pd.read_csv(f)
        lst.append(df)
        big_df = pd.concat(lst, axis = 0, keys=["first", "second", "third", "fourth", "fifth", "seven"
                                               "eigth", "ninth"])
big_df.to_excel("../data/bigdf.xlsx")
big_df

In [318]:
# I want to calculate the mean distances of every category
def calculate_distances(category):
    means = []
    counts = []
    path = os.getcwd()

    for row in range(len(companies)):
        csv_files = glob.glob(os.path.join("../data/", f"{row}{category}.csv"))
        for f in csv_files:
            if os.path.getsize(f) < 100:
                display(f)
            elif os.path.getsize(f) > 100:
                df = pd.read_csv(f)

                means.append(df["distance"].mean())
                counts.append((df["distance"].count()))
    pass

In [329]:
calculate_distances("tram")

'../data/1tram.csv'

'../data/2tram.csv'

'../data/3tram.csv'

'../data/6tram.csv'

'../data/11tram.csv'

In [308]:
means.insert(1, 0)
means.insert(2, 0)
means.insert(3, 0)
means.insert(6, 0)
means.insert(11, 0)
counts.insert(1, 0)
counts.insert(2, 0)
counts.insert(3, 0)
counts.insert(6, 0)
counts.insert(11, 0)

companies["Basket court mean distance"] = means
companies["Basket court counts"] = counts

In [314]:
companies

,Name,Number of employees,Description,Total money (in million $),Main address,City,Latitude,Longitude,Vegan mean distance,Vegan counts,...,Airport mean distance,Aiport counts,Bus station mean distance,Bus station counts,Metro station mean distance,Metro station counts,Train station mean distance,Train station counts,Basket court mean distance,Basket court counts
0,ngmoco,150,Games for the iPhone,44.8,185 Berry Street,San Francisco,37.770510,-122.401581,0.000000,0,...,0.0,0,569.100000,10,1737.500000,2,1022.8,10,833.750000,4
1,Thumbplay,70,"Music, Videos, Games for Mobile Devices",41.5,599 Broadway,New York,40.725435,-73.997395,393.111111,9,...,3852.0,1,549.200000,10,0.000000,0,782.3,10,1394.125000,8
2,OMGPOP,50,Free online multiplayer game,16.6,SoHo,New York,40.723384,-74.001704,395.571429,7,...,4193.0,1,658.200000,10,0.000000,0,818.7,10,1183.000000,6
3,CyberArts,55,Gaming software developer,16.3,"51 Arbor St,",San Francisco,37.737609,-122.435399,0.000000,0,...,0.0,0,957.300000,10,0.000000,0,1010.9,10,1641.500000,2
4,Zynga,115,Network of gaming applications,8.0,365 Vermont St.,San Francisco,37.765158,-122.404234,137.000000,1,...,0.0,0,437.300000,10,1775.000000,1,1331.0,10,1102.250000,4
5,Serious Business,22,Social Game Development Studio,4.0,580 Market Street,San Francisco,37.789321,-122.401362,352.500000,4,...,0.0,0,248.000000,10,742.250000,4,618.9,10,1716.666667,3
6,Exent,100,Games on Demand Leader,3.0,632 W. 28th St.,New York,40.752380,-74.005568,379.000000,3,...,7571.0,2,744.200000,10,0.000000,0,939.8,10,1539.800000,5
7,Exent,100,Games on Demand Leader,3.0,685 Market Street,San Francisco,37.787646,-122.402759,376.000000,4,...,0.0,0,268.000000,10,797.750000,4,603.2,10,1539.000000,3
8,GameLayers,5,Multiplayer Networked Games,2.0,1 Lombard Street,San Francisco,37.803847,-122.402468,0.000000,0,...,0.0,0,885.000000,10,1374.600000,5,712.8,10,1583.000000,1
9,Shattered Reality Interactive,10,"Online Social Gaming, MMO",1.1,490 2nd Street,San Francisco,37.783046,-122.394067,301.000000,1,...,0.0,0,446.100000,10,1079.000000,3,880.3,10,1244.666667,3


In [315]:
companies.to_excel("../data/companiesfull.xlsx")